# Fitting a stretched CPMG function to the relaxation dispersion spectra derived from chemical shifts of MD trejcotry frames

## Steps

0. Predict chemical shifts from the MD trajecotry frames using e.g. [SPARTA+](https://doi.org/10.1007/s10858-010-9433-9).
1. Calcualte the RD spectra using Fourier transformation with proper normalization.
2. Fit a stretched CPMG function using [pymc](https://doi.org/10.7717/peerj-cs.1516)


## Theory 

RD profiles were calculated from our MD simulations via the power spectrum of the combined chemical shift time traces CSHN(t), calculated for all available trajectories of the respective nuclei. Chemical shifts of the p53-TAD backbone protons were predicted using the software SPARTA+ (91) from simulation frames separated 100 ps in time from 20 μs trajectories for the WT and 60 μs trajectories for the P27A system. As shown by Xue et al. earlier, the RD spectrum is derived from the chemical shift autocorrelation functions (94, 95), which we have calculated from CSHN(t) via the properly normalized power spectrum, i.e., the absolute-squared Fourier transform

$ R_{2,ex}\left(\nu\right)\frac{1}{2}N\Delta t=\left|\mathcal{F}\left(CS_{NH}\left(t\right)\right)\right|^2   $

using the Wiener-Khinchin theorem (96). Here, 𝜈 is the CPMG frequency, N is the number of simulation frames, Δt is the time step between frames (100 ps). Note that the unit of CSHN(t) is rad/s. RD profiles were calculated separately from each MD trajectory and then averaged.

The analytical form of CPMG profiles for a two-state model (44, 100) is

$ R_{2,MD}\left(\nu\right)=\phi\tau\left(1-4\nu\tau\ tanh\frac{1}{4\nu\tau}\right) $

Here, 𝜈 is the CPMG frequency, Φ = pA pB ΔCS2 is the population weighted chemical shift variance, pA and pB are the populations of the two states A and B of the model, respectively, and ΔCS is the chemical shift difference between these two states.
Inspired by the shape of the RD profiles calculated from the simulations, which in the logarithmic plot appears to be ‘stretched’ in the frequency domain relative to the CPMG two-state model, a generalized model in terms of a ‘stretched’ function was considered,

$ R_{2,MD}\left(\nu\right)=\phi\tau\left(1-\left(4\nu\tau\right)^\gamma\ tanh\frac{1}{\left(4\nu\tau\right)^\gamma}\right) $

similar in spirit to the ‘stretched exponential functions’ used by Frauenfelder to describe the multi-tier dynamics of folded proteins (5, 101, 102). The function can be interpreted as a weighted superposition of many two-state CPMG profiles, with a broad range of characteristic timescales, which is described by the additional fitting parameter 𝛾. Accordingly, 𝛾 also characterizes the distribution width of barrier heights governing p53-TAD dynamics, namely 𝛾 = 1 indicate a two-state process and the smaller the value the broader the timescale distribution is.

Fitting was perfomred using a Bayesian approach (104) with the pymc5 Python package (105), with 𝜏 and Φ and, 𝛾 as free parameters to be determined. The fact that all RD profiles were calculated by averaging over 30 individual RD profiles, each calculated from an absolute-squared Fourier transform of chemical shift trajectories, required particular attention. Notably, Fourier transforms calculated numerically from finite time series are notoriously noisy, and the distribution of each individual (absolute squared) Fourier coefficient $ R_k := R_2(𝜈_k) $ for realizations with a uniform distribution of random phases follows an exponential function (106),

$ p\left(R_k\right)=\frac{1}{R_k^0}e^{-\frac{R_k}{R_k^0}} $

where $ p(R_k^0) $ is the true coefficient from which the realizations were drawn. Hence, the probability distribution of the mean $ {\bar{R}}_k $ of N=30 Fourier transforms of realizations of the same process (the phases of which are assumed to be statistically independent and uniformly distributed) follows a gamma distribution

$ p\left({\bar{R}}_k\right)=\frac{\beta^N{\bar{R}}_k^{N-1}e^{-\beta{\bar{R}}_k}}{\Gamma\left(N\right)} $

where  $ \beta=\frac{N}{{\bar{R}}_k} $

Accordingly, this probability distribution was used (instead of a Gaussian distribution) for the Bayesian fit of each Fourier coefficient, with the fitting target $ {\bar{R}}_k $. A log-uniform prior distribution between $ 10^3 -10^9 Hz^2 $ was used for Φ, a log-uniform prior between 1 ns-100 μs for 𝜏, and a uniform prior between 0-2 for 𝛾.

Note: references follow the numbering of the paper.

## Calculation of the RD spectra


In [9]:
import numpy as np
import glob
import pymc as pm 
import pandas as pd
from scipy.fft import fft, fftfreq

# input file name stem (used with glob to get a list of chemical shift files)
infile = "/path/to/chemical_shift_files_*.dat"
# Larmor frequency of the NMR experiment in MHz
Larmor = 1200.236
#time step of chemical shift time traces in seconds
dt = 1e-10
#the number of points considered for the fitting, starting with the lowest frequency. 
wmax = 500
#output path stem
out_path = "/path/to/output_"

#define the fitted CPMG function
def CPMG_stretched(nu,Phi,tau,stretch_factor):
    return Phi*tau * (1 - (4*nu*tau)**stretch_factor * np.tanh(1 / (4*nu*tau)**stretch_factor))

In [2]:
#chemical shft file list
CS_file_list = glob.glob(infile)

#determine the length of the files
with open(CS_file_list[0]) as f:
    N = sum(1 for _ in f)

#determine the frequencies, 𝜈
nu = np.arange(N) / (dt*N)

#contained for the fft results, maximum frequency  points × trajectories
fft_MD = np.zeros((wmax,len(CS_file_list))) 

#iterate over the time traces
for i,F in enumerate(CS_file_list):
    CS = np.loadtxt(F,max_rows=N, usecols=1)
    
    fftemp = fft(CS*2*np.pi*Larmor,norm="forward")
    fft_MD[:,i] = np.abs(fftemp[:wmax]) * np.abs(fftemp[:wmax])

#normalize the power spectra to get the RD of each trace.
fft_MD = 0.5 * dt * N * fft_MD # do all the scaling 


In [ ]:
#get the sum of the traces which is going to be fitted
R2 = np.sum(fft_MD[1:,:],axis=1) 
#limit the frequency range
nu = nu[np.arange(1,wmax)]

#set up pymc model and sample
with pm.Model() as model_MD_stretched:
    #log-uniform prior
    Phi_power = pm.Uniform('Phi_power', 3, 9)
    Phi = pm.Deterministic('Phi', 10**Phi_power)
    #power for loguniform tau
    tau_power = pm.Uniform('tau_power', -9, -4) 
    tau = pm.Deterministic('tau',10**tau_power)
    stretch_factor = pm.Uniform('stretch_factor', 0, 2)

    #parameters for gamma distribution
    alpha = len(CS_file_list) # number of fft summed
    beta = alpha / CPMG_noR0_stretched(nu, Phi, tau, stretch_factor)
    sigma = pm.math.sqrt(alpha / (beta**2))

    #the likelyhood
    R2_pred = pm.Gamma('R2_pred', mu=CPMG_noR0_stretched(nu, Phi, tau, stretch_factor) , sigma=sigma, observed=R2)
    #sampling
    trace_MD_stretched = pm.sample(2000, tune=1000,chains=4, cores=4, return_inferencedata=True,target_accept=0.99)


## extract parameters and save

In [ ]:

#----- take from all bootstrap round the last 1000 samples and put them into a big dataframe
#                                               samples*chains 
output_df = pd.DataFrame(np.zeros((bootstrap_rounds*1000*4,4)), columns=["Phi","tau","stretch_factor"])

Phis = []
taus = []
stretch_factors = []

for j,trace_MD_stretched in enumerate(trace_list):
    #Phi is scaled to get the mean parameter for the mean traces instead of the sum
    Phis.append(np.ravel(trace_MD_stretched.posterior["Phi"].values[:,1000:]) / len(CS_file_list) ) # take only last half of sampling, scaled by number of repeats
    taus.append(np.ravel(trace_MD_stretched.posterior["tau"].values[:,1000:]))
    stretch_factors.append(np.ravel(trace_MD_stretched.posterior["stretch_factor"].values[:,1000:]))
           
output_df["Phi"] = np.concatenate(Phis)
output_df["tau"] = np.concatenate(taus)
output_df["stretch_factor"] = np.concatenate(stretch_factors)

#save data frame of parameters
output_df.to_pickle(out_path+"fit_parameter_trace.gz", compression='infer', protocol=5)
